# Basis Functions

In [ ]:
# import numpy for the arrays
import numpy as np

# ensure plots appear in the notebook
%matplotlib inline 
import pylab as plt

import pods
data = pods.datasets.olympic_marathon_men()
y = data['Y']
x = data['X']
y -= y.mean()
y /= y.std()

In [ ]:
def radial(x, num_basis=4, data_limits=[-1., 1.]):
    "Radial basis constructed using exponentiated quadratic form."
    if num_basis>1:
        centres=np.linspace(data_limits[0], data_limits[1], num_basis)
        width = (centres[1]-centres[0])/2.
    else:
        centres = np.asarray([data_limits[0]/2. + data_limits[1]/2.])
        width = (data_limits[1]-data_limits[0])/2.
    
    Phi = np.zeros((x.shape[0], num_basis))
    for i in range(num_basis):
        Phi[:, i:i+1] = np.exp(-0.5*((x-centres[i])/width)**2)
    return Phi

In [ ]:
def fourier(x, num_basis=4, data_limits=[-1., 1.]):
    "Fourier basis"
    tau = 2*np.pi
    span = float(data_limits[1]-data_limits[0])
    Phi = np.zeros((x.shape[0], num_basis))
    for i in range(num_basis):
        count = float((i+1)//2)
        frequency = count/span
        if i % 2:
            Phi[:, i:i+1] = np.sin(tau*frequency*x)
        else:
            Phi[:, i:i+1] = np.cos(tau*frequency*x)
    return Phi

In [ ]:
def polynomial(x, num_basis=4, data_limits=[-1., 1.]):
    "Polynomial basis"
    centre = data_limits[0]/2. + data_limits[1]/2.
    span = data_limits[1] - data_limits[0]
    z = x - centre
    z = 2*z/span
    Phi = np.zeros((x.shape[0], num_basis))
    for i in range(num_basis):
        Phi[:, i:i+1] = z**i
    return Phi

### Question 3: Hand Fitting

In [ ]:
# Question 3 Answer Code
# provide the answers so that the code runs correctly otherwise you will loose marks!

# (a) polynomial
###### Edit these lines #####
w_0 =-0.49
w_1 = -0.81
w_2 = 1.735
w_3 =-1.68
##############################
w_polynomial = np.asarray([[w_0], [w_1], [w_2], [w_3]]) 

# (b) rbf
###### Edit these lines #####
w_0 =2.788
w_1 = -0.117
w_2 = -0.572
w_3 =-0.809
##############################
w_rbf = np.asarray([[w_0], [w_1], [w_2], [w_3]]) 

# (c) fourier
###### Edit these lines #####
w_0 =0.761
w_1 = -0.339
w_2 = -0.066
w_3 =0.163
##############################
w_fourier = np.asarray([[w_0], [w_1], [w_2], [w_3]]) 

### Question 4: Linear or not Linear

(a) $f(x) = w_1x_1 + w_2$
- linear in inputs
- linear in parameters

(b) $f(x) = w_1\exp(x_1) + w_2x_2 + w_3$
- not linear in inputs
- linear in parameters
    
(c) $f(x) = \log(x_1^{w_1}) + w_2x_2^2 + w_3 = w_1 \log x_1 + w_2 x_2^2 + w_3$
- not linear in inputs
- linear in parameters

(d) $f(x) = \exp(-\sum_i(x_i - w_i)^2)$
- not linear in inputs
- not linear in parameters

(e) $f(x) = \exp(-\mathbf{w}^\top \mathbf{x})$
- not linear in inputs
- not linear in parameters

### Question 5: Sample Code

In [ ]:
# Question 5 Answer Code
# Write code for you answer to this question in this box
# Do not delete these comments, otherwise you will get zero for this answer.
# Make sure your code has run and the answer is correct *before* submitting your notebook for marking.

# Compute design matrix
Phi = polynomial(x, num_basis=4, data_limits=[1880, 2020])

# Solve the linear system for the model parameters
w = np.linalg.solve(np.dot(Phi.T, Phi), np.dot(Phi.T, y))

# Compute the error
err_linalg = ((y - [np.dot(phi, w) for phi in Phi])**2).sum()
print("Linear algebra (OLS) error: ", err_linalg)

# Form the prediction function
pred = [np.dot(phi, w).sum() for phi in Phi]

# Compute the error of the function fitted by hand
err_byhand = ((y - [np.dot(phi, w_polynomial) for phi in Phi])**2).sum()
print("By hand error:              ", err_byhand)

# The prediction function fitted by hand
pred_hand = [np.dot(phi, w_polynomial).sum() for phi in Phi]

# Plot
plt.plot(x, pred, label="Prediction Function: Linear Algebra (OLS)")
plt.plot(x, pred_hand, label="Prediction Function: By Hand")
plt.plot(x, y, 'rx')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()